In [1]:
import numpy as np

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def relu(x):
    return np.maximum(0, x)

def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis = 0)
        y = np.exp(x) / np.sum(np.exp(x), axis = 0)

        return y.T

    x = x - np.max(x)
    exp_x = np.exp(x)

    return exp_x / np.sum(exp_x)

def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshpae(1, t.size)
        y = y.reshape(1, y.size)

    batch_size = y.shape[0]
    return -np.sum(t * np.log(y + 1e-7)) / batch_size

def numerical_gradient(f, x):
    h = 10e-4

    return (f(x + h) - f(x - h)) / (2 * h)

In [3]:
import numpy as np

#relu関数のクラス
class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx

#sigmoid関数のクラス
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = sigmoid(x)
        self.out = out

        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out

        return dx

#それぞれの層になるクラス
class Affine:
    def __init__(self, w, b):
        self.w = w
        self.b = b

        self.x = None
        self.dw = None
        self.db = None

    def forward(self, x):
        self.x = x
        out = np.dot(self.x, self.w) + self.b

        return out

    def backward(self, dout):
        dx = np.dot(dout, self.w.T)
        self.dw = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis = 0)

        return dx

#ソフトマックス関数と交差エントロピー誤差をまとめたクラス
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)

        return self.loss

    def backward(self, dout = 1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size

        return dx

In [4]:
import numpy as np

class NeuralNet:
    #レイヤーを保持するためのリストと重みパラメータの初期値を調整する値を持つ
    def __init__(self, weight_init_std = 0.01):
        #レイヤを保持するリスト
        self.layers = []
        #アフィンレイヤのみを保持するリスト
        self.affineLayers = []
        #最後のレイヤ(出力層)、今回はSoftmax
        self.lastLayer = SoftmaxWithLoss()
        #重みの初期値を決めるための変数
        self.weight_init_std = weight_init_std

    #アフィンレイヤを追加する
    def add_affine(self, input_size, output_size):
        w = np.random.randn(input_size, output_size) * self.weight_init_std
        b = np.zeros(output_size)
        layer = Affine(w, b)
        self.layers.append(layer)
        self.affineLayers.append(layer)

    #活性化関数を追加する
    def add_active(self, func_name):
        if func_name == "relu":
            self.layers.append(Relu())
        elif func_name == "sigmoid":
            self.layers.append(Sigmoid())

    #予測を行う（入力層から隠れ層までの処理をする）
    def predict(self, x):
        for layer in self.layers:
            x = layer.forward(x)

        return x

    #損失関数
    def loss(self, x, t):
        y = self.predict(x)

        return self.lastLayer.forward(y, t)

    #予測精度を求める
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis = 1)
        if t.ndim != 1:
            t = np.argmax(t, axis = 1)

        accuracy = np.sum(y == t) / x.shape[0]

        return accuracy

    #勾配を数値微分によって求める
    def numerical_gradient(self, x, t):
        loss_W = lambda W : self.loss(x, t)

        for layer in self.affineLayers:
            layer.dw = numerical_gradient(loss_W, layer.w)
            layer.db = numerical_gradient(loss_W, layer.b)

    #逆伝播による勾配計算を行う
    def gradient(self, x, t):
        self.loss(x, t)
        dout = 1
        dout = self.lastLayer.backward(dout)
        layers = list(self.layers)
        #逆伝播なので後ろのレイヤから勾配を伝播させる
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

    #学習を行う
    def fit(self, x, t, learning_rate = 0.1):
        self.gradient(x, t)

        for layer in self.affineLayers:
            layer.w -= learning_rate * layer.dw
            layer.b -= learning_rate * layer.db

In [6]:
import sys, os
sys.path.append(os.pardir)

import numpy as np
from dataset.mnist import load_mnist

#mnistを読み込む
(x_train, t_train), (x_test, t_test) = load_mnist(normalize = True, one_hot_label = True)

model = NeuralNet()

#各設定値
iters_num = 5000 #繰り返す回数
train_size = x_train.shape[0] 
batch_size = 100 #バッチ（データの塊）の数
learning_rate = 0.1 #学習率

iter_per_epoch = max(train_size // batch_size, 1)

train_loss_list = []
train_acc_list = []
test_acc_list = []

model.add_affine(784, 50)
model.add_active("relu")
model.add_affine(50, 10)

for i in range(iters_num):
    #ランダムに取り出してバッチを生成
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    model.fit(x_batch, t_batch, learning_rate)

    loss = model.loss(x_batch, t_batch)
    train_loss_list.append(loss)

    if i % iter_per_epoch == 0:
        train_acc = model.accuracy(x_train, t_train)
        test_acc = model.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(str(i) + "回目")
        print(train_acc, test_acc)

0回目
0.13543333333333332 0.1358
600回目
0.90455 0.9068
1200回目
0.9248166666666666 0.9276
1800回目
0.9359166666666666 0.9356
2400回目
0.9454 0.9445
3000回目
0.9496 0.9476
3600回目
0.9567833333333333 0.9538
4200回目
0.9607166666666667 0.9574
4800回目
0.9642166666666667 0.9618
